# Hetero-NN Customize Models & Loss

We can customize the top model and the bottom model in the Hetero NN.
The model_zoo module was introduced in FATE 1.10, located under federatedml.nn.model_zoo. This module allows you to customize your own PyTorch model, provided that it is developed based on torch.nn.Module and implements the forward interface. For more information, see the PyTorch documentation  [Pytorch Module](https://pytorch.org/docs/stable/notes/modules.html#a-simple-custom-module) on custom modules. To use your custom model in a federated task, simply place it in the federatedml/nn/model_zoo directory and specify the module and model class through the interface when submitting the task. Hetero-NN components will automatically search and import the model you have implemented.

You can also define your own loss class in a similar way. The you can place your loss class under the loss module, located under federatedml.nn.loss.
We recommend you read these two tutorials before reading this tutorial: [Customize loss function](Homo-NN-Customize-Loss.ipynb), [Customize Model](Homo-NN-Customize-Model.ipynb)

As an example, we consider reuse the task of MNIST handwriting recognition of last hetero-nn tutorial.

## Prepare MNIST Data

Please download the guest/host MNIST dataset from the link below and place it in the project examples/data folder:

- guest data: https://webank-ai-1251170195.cos.ap-guangzhou.myqcloud.com/fate/examples/data/mnist_guest.zip

- host data: https://webank-ai-1251170195.cos.ap-guangzhou.myqcloud.com/fate/examples/data/mnist_host.zip
  
The mnist_guest is a simplified version of the MNIST dataset, with a total of ten categories, which are classified into 0-9 10 folders according to labels. The mnist_host has the same images as the mnist_guest, but it is not labeled.

## Dataset

In version FATE-1.10, FATE introduces a new base class for datasets called Dataset, which is based on PyTorch's Dataset class. This class allows users to create custom datasets according to their specific needs. The usage is similar to that of PyTorch's Dataset class, with the added requirement of implementing two additional interfaces when using FATE-NN for data reading and training: load() and get_sample_ids().

To create a custom dataset in Hetero-NN, users need to:

- Develop a new dataset class that inherits from the Dataset class
- Implement the \_\_len\_\_() and \_\_getitem\_\_() methods, which are consistent with PyTorch's Dataset usage. The \_\_len\_\_() method should return the length of the dataset, while the \_\_getitem\_\_() method should return the corresponding data at the specified index. **However, please notice that different \_\_getitem\_\_() methods may have different behaviors between different parties. In the guest party(party with labels), _\_getitem\_\_() method return features and labels, while in the host parties(parties without label), _\_getitem\_\_() method return features only.** 
- Implement the load(), get_sample_ids(), get_classes() methods
  
For those unfamiliar with PyTorch's Dataset class, more information can be found in the PyTorch documentation: [Pytorch Dataset Documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

## Customize Bottom/Top Model

Name the model code bottom_net.py,  you can put it directly under federatedml/nn/model_zoo or use the shortcut interface of jupyter notebook: save_to_fate, to save it directly to federatedml/nn/model_zoo. This is the bottom model structure we define for feature extraction.

In [1]:
from pipeline.component.nn import save_to_fate

/data/projects/fate/env/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
%%save_to_fate model ad_bottom_host.py
import torch as t
from torch import nn
from torch.nn import Module
import logging
import numpy as np

class BottomNet(nn.Module):

    def __init__(self):
        super(BottomNet, self).__init__()
        self.cuda = True
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )

        self.fc = t.nn.Sequential(   # extracted feature is a 8-dim embedding
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = x[0]
        x = self.seq(x)
        x = x.mean((2,3))
        #x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x

And this is the top model we define for classification, we named it as top_model.py.

In [6]:
%%save_to_fate model ad_bottom_guest.py
import torch as t
from torch import nn
from torch.nn import Module

class DummyNet(nn.Module):

    def __init__(self):
        super(DummyNet, self).__init__()
        self.fc = t.nn.Linear(12,10)

    def forward(self, x):
        x = self.fc(x)
        return x

In [7]:
%%save_to_fate model ad_top.py
import torch as t
from torch import nn
from torch.nn import Module

class TopNet(nn.Module):

    def __init__(self):
        super(TopNet, self).__init__()
        self.fc = t.nn.Sequential(   
            nn.Linear(10, 4),
            t.nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

## Use Cust Loss

Using Cust Loss is exactly the same as Homo-NN, see: [Customize loss function](Homo-NN-Customize-Loss.ipynb). Here we use a new CrossEntropyLoss.

In [57]:
%%save_to_fate loss ce.py
import torch as t
from federatedml.util import consts
from torch.nn.functional import one_hot
from fate_arch.common import log

LOGGER = log.getLogger()

def cross_entropy(p2, p1, reduction='mean'):
    p2 = p2 + consts.FLOAT_ZERO  # to avoid nan
    assert p2.shape == p1.shape
    if reduction == 'sum':
        return -t.sum(p1 * t.log(p2))
    elif reduction == 'mean':
        return -t.mean(t.sum(p1 * t.log(p2), dim=1))
    elif reduction == 'none':
        return -t.sum(p1 * t.log(p2), dim=1)
    else:
        raise ValueError('unknown reduction')


class CrossEntropyLoss(t.nn.Module):

    """
    A CrossEntropy Loss that will not compute Softmax
    """

    def __init__(self, reduction='mean'):
        super(CrossEntropyLoss, self).__init__()
        self.reduction = reduction

    def forward(self, pred, label):
        LOGGER.info(label.shape)
        LOGGER.info(label[0])
        one_hot_label = one_hot(label.to(t.int64))
        LOGGER.info(one_hot_label.shape)
        one_hot_label = t.flatten(one_hot_label, start_dim=1)
        LOGGER.info(one_hot_label.shape)
        LOGGER.info(one_hot_label[0])
        LOGGER.info("pred")
        LOGGER.info(pred.shape)
        LOGGER.info(pred[0])
        
        loss_ = cross_entropy(pred, one_hot_label, self.reduction)

        return loss_

Then, we can use our models & loss in the Hetero-NN MNIST task! The usage is the same as Homo-NN: we specify our model and loss by nn.CustModel and nn.CustLoss interfaces.

## pipeline initialization

Here we define the pipeline to run a hetero task

In [47]:
import os
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HeteroNN
from pipeline.component.hetero_nn import DatasetParam
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform, Intersection
from pipeline.interface import Data, Model
from pipeline.component.nn import save_to_fate

fate_torch_hook(t)

# bind path to fate name&namespace
fate_project_path = os.path.abspath('/data/projects/fate')
guest = 10000
host = 9999

pipeline_img = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

In [48]:
guest_train = {"name": "ad_train_guest", "namespace": "experiment"}
host_train = {"name": "ad_train_host", "namespace": "experiment"}

guest_train_path = fate_project_path + '/examples/data/oasis2train/oasis2.csv'
host_train_path = fate_project_path + '/examples/data/oasis2train/mri/'
pipeline_img.bind_table(name=guest_train['name'], namespace='experiment', path=guest_train_path)
pipeline_img.bind_table(name=host_train['name'], namespace='experiment', path=host_train_path)

guest_val = {"name": "ad_val_guest", "namespace": "experiment"}
host_val = {"name": "ad_val_host", "namespace": "experiment"}

guest_val_path = fate_project_path + '/examples/data/oasis2val/oasis2.csv'
host_val_path = fate_project_path + '/examples/data/oasis2val/mri/'
pipeline_img.bind_table(name=guest_val['name'], namespace='experiment', path=guest_val_path)
pipeline_img.bind_table(name=host_val['name'], namespace='experiment', path=host_val_path)

guest_test = {"name": "ad_test_guest", "namespace": "experiment"}
host_test = {"name": "ad_test_host", "namespace": "experiment"}

guest_test_path = fate_project_path + '/examples/data/oasis2test/oasis2.csv'
host_test_path = fate_project_path + '/examples/data/oasis2test/mri/'
pipeline_img.bind_table(name=guest_test['name'], namespace='experiment', path=guest_test_path)
pipeline_img.bind_table(name=host_test['name'], namespace='experiment', path=host_test_path)

{'namespace': 'experiment', 'table_name': 'ad_test_host'}

In [49]:
#guest_data = {"name": "ad_guest", "namespace": "experiment"}
#host_data = {"name": "ad_host", "namespace": "experiment"}
reader_train = Reader(name="reader_train")
reader_train.get_party_instance(role='guest', party_id=guest).component_param(table=guest_train)
reader_train.get_party_instance(role='host', party_id=host).component_param(table=host_train)

reader_val = Reader(name="reader_val")
reader_val.get_party_instance(role='guest', party_id=guest).component_param(table=guest_val)
reader_val.get_party_instance(role='host', party_id=host).component_param(table=host_val)

reader_test = Reader(name="reader_test")
reader_test.get_party_instance(role='guest', party_id=guest).component_param(table=guest_test)
reader_test.get_party_instance(role='host', party_id=host).component_param(table=host_test)

In [50]:
hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=20,
                       interactive_layer_lr=0.01, batch_size=32, task_type='classification', seed=100
                       )
guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)

# define model
# use cust model here
# our simple classification model:

guest_bottom = t.nn.CustModel(module_name='ad_bottom_guest.py', class_name='DummyNet')
#guest_bottom = None

# use cust model here
host_bottom = t.nn.CustModel(module_name='ad_bottom_host.py', class_name='BottomNet')

# use new top model here
guest_top = t.nn.CustModel(module_name='ad_top.py', class_name='TopNet')

# interactive layer define
#interactive_layer = t.nn.InteractiveLayer(out_dim=8, guest_dim=None, host_dim=8)
interactive_layer = t.nn.InteractiveLayer(out_dim=10, guest_dim=10, host_dim=10)

# add models
guest_nn_0.add_top_model(guest_top)

guest_nn_0.add_bottom_model(guest_bottom)

host_nn_0.add_bottom_model(host_bottom)

# opt, loss
optimizer = t.optim.Adam(lr=0.01) 
loss = t.nn.CustLoss(loss_module_name='ce', class_name='CrossEntropyLoss')
#loss = t.nn.CrossEntropyLoss()

# use DatasetParam to specify dataset and pass parameters
host_nn_0.add_dataset(DatasetParam(dataset_name='image', return_label=True))
guest_nn_0.add_dataset(DatasetParam(dataset_name='table'))

hetero_nn_0.set_interactive_layer(interactive_layer)
hetero_nn_0.compile(optimizer=optimizer, loss=loss)

In [51]:
pipeline_img.add_component(reader_train)
pipeline_img.add_component(reader_val)
pipeline_img.add_component(reader_test)
pipeline_img.add_component(hetero_nn_0, data=Data(train_data=reader_train.output.data, validate_data = reader_val.output.data))
pipeline_img.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=hetero_nn_0.output.data))
pipeline_img.compile()


In [52]:
pipeline_img.fit()

2023-11-27 05:59:42.604 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202311270559423279260

2023-11-27 05:59:42.611 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-11-27 05:59:43.636 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-11-27 05:59:44.651 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-11-27 05:59:44.652 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_train, time elapse: 0:00:02
2023-11-27 05:59:45.666 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_train, time elapse: 0:00:03
2023-11-27 05:59:46.682 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_train, time elapse: 0:00:04
2023-11-27 05:59:47.706 | INFO

In [55]:
df = pipeline_img.get_component('hetero_nn_0').get_output_data()  # get result

In [54]:
pipeline_img.get_component('hetero_nn_0').get_summary()

{'best_iteration': -1,
 'history_loss': [0.952840482487398,
  0.9338948426126432,
  0.9078122292246137,
  0.7935138535098869,
  0.5299976690977561,
  0.3717174545055678,
  0.32122436441293284,
  0.2989905430238788,
  0.2866426619912396,
  0.27128627971440805,
  0.2586645312419459,
  0.24963022743453497,
  0.2491156666719613,
  0.2447387887900617,
  0.23895372475395685,
  0.2348941915175494,
  0.23198429188307593,
  0.22975258003262913,
  0.22803287483563944,
  0.22645890299512558],
 'is_converged': False}

In [62]:
!flow component output-data -j 202311270559423279260 -r guest -p 10000 -cpn hetero_nn_0 --output-path ./

{
    "retcode": 0,
    "directory": "/data/projects/fate/doc/tutorial/vertical_alzheimer/job_202311270559423279260_hetero_nn_0_guest_10000_output_data",
    "retmsg": "Download successfully, please check /data/projects/fate/doc/tutorial/vertical_alzheimer/job_202311270559423279260_hetero_nn_0_guest_10000_output_data directory"
}



In [10]:
#pipeline_img.deploy_component([hetero_nn_0])
pipeline_img.deploy_component([intersection_0,hetero_nn_0])

In [9]:
reader_test = Reader(name="reader_test")
reader_test.get_party_instance(role='guest', party_id=guest).component_param(table=guest_val)
reader_test.get_party_instance(role='host', party_id=host).component_param(table=host_val)

In [10]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_train)
predict_pipeline.add_component(pipeline_img, 
                               data=Data(predict_input={pipeline_img.hetero_nn_0.input.data: reader_train.output.data}))
predict_pipeline.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=pipeline_img.hetero_nn_0.output.data))

2023-11-27 05:39:33.915 | ERROR    | __main__:<module>:3 - An error has been caught in function '<module>', process 'MainProcess' (446), thread 'MainThread' (140123729659712):
Traceback (most recent call last):

  File "/data/projects/fate/env/python/miniconda/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7f71181a20e0, file "/data/projects/fate/env/python/venv/lib/python3.8/site-packages/ipykernel_laun...
           └ <function _run_code at 0x7f71181794c0>
  File "/data/projects/fate/env/python/miniconda/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
  

ValueError: To use a training pipeline object as predict component, should deploy model first

In [39]:
reader_train

In [40]:
predict_pipeline.predict()

2023-11-27 05:29:47.672 | ERROR    | pipeline.backend.pipeline:predict:638 - An error has been caught in function 'predict', process 'MainProcess' (8087), thread 'MainThread' (140342172002112):
Traceback (most recent call last):

  File "/data/projects/fate/env/python/miniconda/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7fa3f447c0e0, file "/data/projects/fate/env/python/venv/lib/python3.8/site-packages/ipykernel_laun...
           └ <function _run_code at 0x7fa3f44534c0>
  File "/data/projects/fate/env/python/miniconda/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ip

AttributeError: 'TrainingInputData' object has no attribute 'split'